![Xilinx Logo](images/xilinx_logo.png "Xilinx Logo")

# 1. Introduction

This notebook demonstrates how to split four video pipelines from a single source. Each branch runs through a 2d filter. This notebook uses 2d filter kernels available in PL HW. The kernels is time-multiplexed across four elements. The display device contains a video mixer which allows targeting different video planes for the four branches with programmable x/y-offsets as well as width and height. In addition, the memory bandwidth is measured and plotted in a parallel notebook.

Five types of V4L2 devices are supported in this notebook:
* Virtual Video Test driver (vivid)
* USB Video Class (UVC) driver (usb)
* MIPI CSI-2 capture pipeline using the Leopard IMX274 FMC daughter card (mipi)
* MIPI CSI-2 capture pipeline using the Avnet Multi-Camera FMC Module (mipi_quad)
* HDMI Rx HDR10 capture pipeline (hdmi)

The video pipeline is composed of the following GStreamer elements:
* The ``mediasrcbin`` element is used to capture video from a V4L2 device
* The ``tee`` element is used to fork the input stream into multiple output streams, in this case 4 ouput streams
* The ``ivas_xfilter`` vvas infrastructure plugin used to implement a 2D convolution filter. A total of 4 instances is used using the PL kernel. 
* The ``perf`` element is used to measure and print the frame rate in one of the forked paths.
* The ``kmssink`` element is used to display video on a monitor using the DRM/KMS kernel subsystem. Four planes are used to display the four streams.

The default input video resolution is set to 1280x720, hence the monitor needs to support a minimum resolution of 2560x1440 (or higher).

The ``base-trd-apm`` notebook is executed in parallel to this notebook ro measure and plot the memory bandwidth of the live video pipeline.

In this notebook, you will:
1. Create a GStreamer video pipeline with four branches using the ``parse_launch()`` API
2. Run the ``base-trd-apm`` notebook to measure and plot the memory bandwidth while the video pipeline is running.
3. Create a GStreamer pipeline graph and view it inside this notebook.

# 2. Imports and Initialization

Import all python modules required for this notebook. 

In [ ]:
from IPython.display import Image, display, clear_output
import glob
import subprocess
import pydot
import sys
import os
import gi
gi.require_version('Gst', '1.0')
from gi.repository import GObject, GLib, Gst

This is the Base TRD notebook 7 (nb7).

In [ ]:
nb = "nb7"

Create a directory for saving the pipeline graph as dot file. Set the GStreamer debug dot directory environment variable to point to that directory.

In [ ]:
dotdir = "/home/root/gst-dot/" + nb
!mkdir -p $dotdir
%env GST_DEBUG_DUMP_DOT_DIR = $dotdir

Validate the correct Vitis Overlay is available in the platform for this notebook.

In [ ]:
def xbutil_program_xclbin():
    xclbin = os.path.join("/media/sd-mmcblk0p1", "binary_container_1.xclbin")
    if os.path.exists(xclbin):
        subprocess.run(['xbutil', 'program', '-p', xclbin], check=True)

def xbutil_query_cu(cu):
    proc = subprocess.run(['xbutil', 'query'], capture_output=True, encoding='utf8')
    for line in proc.stdout.splitlines():
        if cu in line:
            return
    raise Exception("Unable to find compute unit \'" + cu + "\'. Make sure the correct Vitis overlay is used.")

xbutil_program_xclbin()
xbutil_query_cu("filter2d_pl_accel_0")

Initialize the GStreamer library. Enable debug by setting the debug string, set default to level 1 for all categories.

In [ ]:
Gst.init(None)
Gst.debug_set_threshold_from_string('*:1', True)

# 3. Run the APM Notebook to Plot the Memory Bandwidth

Open the ``base-trd-apm.ipynb`` notebook from the *File Browser* in a new tab. Execute the notebook by selecting *Run -> Run All Cells* from the Jupyter Lab menu bar. In section 4 of the APM notebook, a horizontal bar graph is shown that plots the currently consumed memory bandwidth split out by different AXI ports. For more information, read the APM notebook tutorial.

Once you see the graph, right-click the graph and select *Create New View for Output*. This will create a new window/tab with just the graph. Now re-arrange the window by dragging it to the the right side of the screen so it shows side-by-side with the notebook window (see screenshot below).

![APM Plot](images/apm-plot-nb6.jpg "APM Plot")

Switch tabs back to the nb7 notebook and follow the steps below. Once the video pipeline is running, you will notice the bar graph will be updated live with the measured memory bandwidth numbers in Gbps. The screenshot shows video capture from the MIPI pipeline.

**Note:** You can skip this step if your APM output view was already created previsouly. It will update automatically after running the video pipeline.

# 4. Create String Representation of GStreamer Pipeline

The ``get_media_by_device`` function returns the matching media node for a given video capture source. The following sources are supported in this notebook:
* ``vivid`` : virtual video device (default)
* ``usb`` : requires USB webcam
* ``mipi`` : platform1 only, requires FMC card
* ``mipi_quad`` : platform2 only, requires FMC card
* ``hdmi`` : platform3 only, requires HDMI input

In [ ]:
def get_media_dev_by_name(src):
    sources = {
        'vivid' : 'vivid',
        "usb" : 'uvcvideo',
        'mipi' : 'vcap_csi',
        'mipi_quad' : 'vcap_gmsl',
        'hdmi' : 'vcap_hdmi'
    }
    devices = glob.glob('/dev/media*')
    for dev in devices:
        proc = subprocess.run(['media-ctl', '-d', dev, '-p'], capture_output=True, encoding='utf8')
        for line in proc.stdout.splitlines():
            if sources[src] in line:
                return dev

Describe the ``mediasrcbin`` element and its properties as string representation.

In [ ]:
source = "vivid" # Change source to vivid, usb, mipi, mipi_quad, or hdmi

media_device = get_media_dev_by_name(source) 
if media_device is None:
    raise Exception('Unable to find video source ' + source + '. Make sure the device is plugged in, powered, and the correct platform is used.')

io_mode = "mmap"
if source == "mipi" or source == "mipi_quad" or source== "hdmi":
    io_mode = "dmabuf"

src = "mediasrcbin media-device=" + media_device + " name=src"

Describe the ``caps`` filter element as string representation. Set the framerate if MIPI or MIPI quad is selected as source.

In [ ]:
width = 1280
height = 720
fmt = "YUY2"

caps = "video/x-raw, width=" + str(width) + ", height=" + str(height) + ", format=" + fmt 

if source == "mipi" or source== "hdmi":
    fps = "60/1"
    caps = caps + ", framerate=" + fps
elif source == "mipi_quad":
    fps = "30/1"
    caps = caps + ", framerate=" + fps

Describe the filter2d elements which can be time-multiplexed across multiple video pipelines. In this case, four instances of the filter2d element are used:
1. Four PL filter2d elements with different filter presets using a single HW kernel. Presets are being dynamically configured. the first preset produces an edge effect (top left), the second is an emboss effect (top right), the third preset produces a sharpen effect (bottom left), the last preset is identity, which takes the default config values from json file.

In [ ]:
filter_pl1 = 'ivas_xfilter kernels-config=/usr/share/ivas/base-trd/kernel_xfilter2d_pl.json dynamic-config={"filter_preset":"edge"}'
filter_pl2 = 'ivas_xfilter kernels-config=/usr/share/ivas/base-trd/kernel_xfilter2d_pl.json dynamic-config={"filter_preset":"emboss"}'
filter_pl3 = 'ivas_xfilter kernels-config=/usr/share/ivas/base-trd/kernel_xfilter2d_pl.json dynamic-config={"filter_preset":"sharpen"}'
filter_pl4 = 'ivas_xfilter kernels-config=/usr/share/ivas/base-trd/kernel_xfilter2d_pl.json'

Describe the ``kmssink`` element and its properties as string representation. Four planes with different x/y-offsets are used to display the four video streams.

In [ ]:
# plane 1
plane_id_1 = 38
xoff_1 = 0
yoff_1 = 0
render_rectangle_1 = "<" + str(xoff_1) + "," + str(yoff_1) + "," + str(width) + "," + str(height) + ">"
sink_1 = "kmssink plane-id=" + str(plane_id_1) + " render-rectangle=" + render_rectangle_1  + " sync=false"
# plane 2
plane_id_2 = 39
xoff_2 = width
yoff_2 = 0
render_rectangle_2 = "<" + str(xoff_2) + "," + str(yoff_2) + "," + str(width) + "," + str(height) + ">"
sink_2 = "kmssink plane-id=" + str(plane_id_2) + " render-rectangle=" + render_rectangle_2 + " sync=false"
# plane 3
plane_id_3 = 40
xoff_3 = 0
yoff_3 = height
render_rectangle_3 = "<" + str(xoff_3) + "," + str(yoff_3) + "," + str(width) + "," + str(height) + ">"
sink_3 = "kmssink plane-id=" + str(plane_id_3) + " render-rectangle=" + render_rectangle_3 + " sync=false"
# plane 4
plane_id_4 = 41
xoff_4 = width
yoff_4 = height
render_rectangle_4 = "<" + str(xoff_4) + "," + str(yoff_4) + "," + str(width) + "," + str(height) + ">"
sink_4 = "kmssink plane-id=" + str(plane_id_4) + " render-rectangle=" + render_rectangle_4 + " sync=false"

Create a string representation of the pipeline by concatenating the individual element strings.

If MIPI quad is selected, ``mediasrcbin`` is instantiated with four source pads, one for each sensor. the source pads are referenced by the name of the ``mediasrcbin`` element (``src``) followed by a dot (``src.``). For any other source, a ``tee`` element is used to fork the source. Each branch is referenced by the name of the ``tee`` element (``src``) followed by a dot (``src.``). The ``queue`` element is used to create a new thread for each branch.

In [ ]:
if source == "mipi_quad":
    t = " src. ! " + caps
else:
    src = src + " v4l2src0::io-mode=" + io_mode + " ! " + caps + " ! tee name=t"
    t = " t. "

pipe = src + \
    t + " ! queue ! " + filter_pl1  + " ! queue ! perf ! " + sink_1 + " " + \
    t + " ! queue ! " + filter_pl2  + " ! queue ! " + sink_2 + " " + \
    t + " ! queue ! " + filter_pl3 + " ! queue ! " + sink_3 + " " + \
    t + " ! queue ! " + filter_pl4 + " ! queue ! " + sink_4

print (pipe)

# 5. Create and Run the GStreamer Pipelines

Parse the string representations of the first and second pipeline as a single pipeline graph.

In [ ]:
pipeline = Gst.parse_launch(pipe)

The ``bus_call`` function listens on the bus for ``EOS``, ``INFO`` and ``ERROR`` events. In case of ``EOS`` or ``ERROR``, stop the pipeline (set to ``NULL`` state) and quit the main loop. 

For ``INFO`` and ``ERROR`` events, parse and print the info/error message. The ``perf`` element generates ``INFO`` events with the measured frame rate.

In [ ]:
def bus_call(bus, message, loop):
    t = message.type
    if t == Gst.MessageType.EOS:
        sys.stdout.write("End-of-stream\n")
        pipeline.set_state(Gst.State.NULL)
        loop.quit()
    elif t == Gst.MessageType.INFO:
        err, info = message.parse_info()
        sys.stderr.write("Info: %s\n" % info)
        clear_output(wait=True)
    elif t == Gst.MessageType.ERROR:
        err, debug = message.parse_error()
        sys.stderr.write("Error: %s: %s\n" % (err, debug))
        pipeline.set_state(Gst.State.NULL)
        loop.quit()
    return True

Start the pipeline (set to ``PLAYING`` state), create the main loop and listen to messages on the bus. Register the ``bus_call`` callback function with the ``message`` signal of the bus. Start the main loop.

The video will be displayed on the monitor. The frame rate will be printed and updated below the code cell.

To stop the pipeline, click the square shaped icon labelled 'Interrupt the kernel' in the top menu bar. Create a dot graph of the pipeline topology before stopping the pipeline. Quit the main loop.

In [ ]:
pipeline.set_state(Gst.State.PLAYING);

loop = GLib.MainLoop()
bus = pipeline.get_bus()
bus.add_signal_watch()
bus.connect("message", bus_call, loop)

try:
    loop.run()
except:
    sys.stdout.write("Interrupt caught\n")
    Gst.debug_bin_to_dot_file(pipeline, Gst.DebugGraphDetails.ALL, nb)
    pipeline.set_state(Gst.State.NULL)
    loop.quit()
    pass

# 6. View the Pipeline dot Graph

Convert the dot file to png and display the pipeline graph. The image will be displayed below the following code cell. Double click on the generate image file to zoom in.

**Note:** This step may take a few seconds.

In [ ]:
dotfile = dotdir + "/" + nb + ".dot"
graph = pydot.graph_from_dot_file(dotfile, 'utf-8')
display(Image(graph[0].create(None, 'png', 'utf-8')))

# 7. Summary

In this notebook you learned how to:
1. Create a GStreamer pipeline with four branches from a string representation using the ``parse_launch()`` API
2. Plot the live memory bandwidth by running the APM notebook in parallel
3. Export the pipeline topology as a dot file image and display it in the notebook

<center>Copyright© 2019 Xilinx</center>